Assignment 1 exercise 2

In [1]:
%env PYTHONHASHSEED=0

env: PYTHONHASHSEED=0


In [2]:
from pyspark import SparkContext
from datetime import datetime
import random
import math

In [3]:
#local spark
sc = SparkContext(appName="Movie plots")
#vocareum
#sc = spark.sparkContext

In [4]:
metadata = sc.textFile("movie.metadata.tsv")

In [5]:
k_gram_size=9

Getting the mapping of movie_id and movie_title

In [6]:
metadata = metadata.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[2]))
format_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
metadata.saveAsTextFile("{0}/{1}".format("title_map", format_time))

In [7]:
metadata.take(10)

[(975900, 'Ghosts of Mars'),
 (3196793, 'Getting Away with Murder: The JonBenét Ramsey Mystery'),
 (28463795, 'Brun bitter'),
 (9363483, 'White Of The Eye'),
 (261236, 'A Woman in Flames'),
 (13696889, 'The Gangsters'),
 (18998739, "The Sorcerer's Apprentice"),
 (10408933, "Alexander's Ragtime Band"),
 (9997961, 'Contigo y aquí'),
 (2345652, 'City of the Dead')]

In [8]:
def get_movie_title(movie_id):
    title=metadata.lookup(movie_id)
    if len(title)!=1:
        return "Custom movie or error"
    else:
        return title[0]

Getting the movie plots to run lsh on

In [9]:
#movie_plots = sc.textFile("plot_summaries.txt")\
#.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[1]))\
#.sample(withReplacement=False, fraction=0.05)

movie_plots = sc.textFile("smaller_plots.txt")\
.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[1]))

In [10]:
movie_plots.take(1)

[(23890098,
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.")]

Function to break a movie's plot to k_gram shingles

In [11]:
def break_k_grams(plot):
    shingles = []
    for i in range(len(plot)-k_gram_size+1):
        shingles.append(plot[i:i+k_gram_size])
        
    return shingles

In [12]:
movie_shingles = movie_plots.mapValues(break_k_grams)

In [13]:
movie_shingles.take(1)


[(23890098,
  ['Shlykov, ',
   'hlykov, a',
   'lykov, a ',
   'ykov, a h',
   'kov, a ha',
   'ov, a har',
   'v, a hard',
   ', a hard-',
   ' a hard-w',
   'a hard-wo',
   ' hard-wor',
   'hard-work',
   'ard-worki',
   'rd-workin',
   'd-working',
   '-working ',
   'working t',
   'orking ta',
   'rking tax',
   'king taxi',
   'ing taxi ',
   'ng taxi d',
   'g taxi dr',
   ' taxi dri',
   'taxi driv',
   'axi drive',
   'xi driver',
   'i driver ',
   ' driver a',
   'driver an',
   'river and',
   'iver and ',
   'ver and L',
   'er and Ly',
   'r and Lyo',
   ' and Lyos',
   'and Lyosh',
   'nd Lyosha',
   'd Lyosha,',
   ' Lyosha, ',
   'Lyosha, a',
   'yosha, a ',
   'osha, a s',
   'sha, a sa',
   'ha, a sax',
   'a, a saxo',
   ', a saxop',
   ' a saxoph',
   'a saxopho',
   ' saxophon',
   'saxophoni',
   'axophonis',
   'xophonist',
   'ophonist,',
   'phonist, ',
   'honist, d',
   'onist, de',
   'nist, dev',
   'ist, deve',
   'st, devel',
   't, develo',
   ', develo

Hashing shingles to integers

In [14]:

movie_shingles = movie_shingles.mapValues(lambda shing: sorted([hash(x)%(2**32) for x in shing]))


movie_shingles.take(2)

[(23890098,
  [990914,
   7120256,
   38145468,
   44560541,
   54011959,
   56356590,
   80420096,
   97640021,
   113395127,
   116371862,
   123046206,
   157264766,
   198895687,
   243832526,
   252803125,
   260081373,
   273605396,
   294085925,
   315539349,
   330413959,
   384515075,
   404071854,
   414681209,
   427907980,
   522436180,
   586928124,
   606193462,
   691753667,
   697714718,
   718484601,
   761426755,
   801210765,
   806927767,
   825306367,
   853086767,
   891018850,
   891189669,
   894158854,
   894599336,
   928837932,
   944567895,
   1040934821,
   1056440024,
   1072119624,
   1194987296,
   1203984477,
   1209761831,
   1227227446,
   1287674285,
   1302937152,
   1321808920,
   1364475058,
   1379072758,
   1403793861,
   1412475303,
   1433571481,
   1501308588,
   1507843798,
   1594448795,
   1602931640,
   1616821336,
   1624273814,
   1672530441,
   1732558524,
   1737834367,
   1754591189,
   1772539095,
   1796020729,
   1818653369,
   18

Random hash function generator

In [15]:
def random_hash():
    a = random.randint(1,1000)
    b = random.randint(0,1000)
    return a,b

In [16]:
random_hash()

(594, 417)

Parameters for LSH

To answer point 2.1, a possible solution is n_bands=75 and n_rows=10 (for sets of 750 signatures), because:

(1-0.8¹⁰)⁷⁵ <= 0.0005 less than 0.5% false negatives with 80% similarity, ensuring at least 99.5% are caught

and

1-(1-0.4¹⁰)⁷⁵ <=0.05 less than 5% false positives with 40% similarity, ensuring no more than 5% are caught

Note: running with these parameters drastically increases execution time, so the notebook is delivered with easier values by default but these need only be changed below

In [17]:
n_bands = 10
n_rows = 10
k=n_bands*n_rows
similarity_threshold = (1/n_bands)**(1/n_rows)
n_shingles = movie_shingles.flatMap(lambda tup: tup[1]).countApproxDistinct()
p = 24000001
n_lsh_buckets = 4999

similarity_threshold

0.7943282347242815

Generating the hash functions

In [18]:
hash_functions = []
for i in range(k):
    hash_functions.append(random_hash())
hash_functions

[(259, 811),
 (503, 590),
 (568, 287),
 (617, 332),
 (395, 754),
 (813, 762),
 (137, 560),
 (28, 879),
 (735, 498),
 (627, 72),
 (930, 145),
 (776, 19),
 (326, 909),
 (529, 998),
 (425, 9),
 (871, 52),
 (421, 57),
 (75, 74),
 (253, 369),
 (385, 317),
 (300, 407),
 (80, 777),
 (903, 254),
 (617, 971),
 (122, 747),
 (315, 268),
 (849, 938),
 (119, 175),
 (310, 575),
 (586, 464),
 (498, 887),
 (594, 731),
 (342, 286),
 (855, 376),
 (822, 553),
 (240, 534),
 (199, 339),
 (170, 680),
 (422, 477),
 (710, 316),
 (934, 741),
 (616, 259),
 (299, 645),
 (467, 305),
 (801, 970),
 (334, 504),
 (948, 394),
 (399, 45),
 (759, 87),
 (784, 9),
 (135, 377),
 (784, 482),
 (953, 592),
 (151, 957),
 (694, 1000),
 (203, 379),
 (766, 606),
 (261, 106),
 (656, 538),
 (783, 776),
 (603, 488),
 (972, 771),
 (168, 72),
 (685, 487),
 (126, 461),
 (236, 521),
 (797, 9),
 (853, 463),
 (428, 449),
 (730, 269),
 (878, 568),
 (486, 249),
 (650, 523),
 (82, 835),
 (537, 202),
 (391, 234),
 (9, 705),
 (341, 746),
 (671

Simple hash function calculation

In [19]:
def calc_hash(a,b,prime,n,value):
    return ((a*value + b)%prime)%n
    

Min hash implementation to turn sets of hashed shingles into signatures

In [20]:
def get_signature(hashed_shingles):
    signature_matrix = {}
    for element in hashed_shingles:
        for i in range(k):
            hash_result = calc_hash(*hash_functions[i],p,n_shingles, element)
            if  hash_result < signature_matrix.get(i,float('inf')):
                signature_matrix[i]=hash_result
    return signature_matrix

In [21]:
movie_signatures = movie_shingles.mapValues(get_signature)
movie_signatures.cache()

PythonRDD[13] at RDD at PythonRDD.scala:53

Function that breaks a set of signatures into bands (n_bands) of rows (n_rows)

In [22]:
def break_signatures_to_bands(signatures):
    set_list = []
    for _ in range(n_bands):
        #set_list.append(set())
        set_list.append([])
        
    for i in signatures:
        #set_list[i//n_rows].add(signatures[i])
        set_list[i//n_rows].append(signatures[i])
    return set_list
    

Function that hashes bands returning a tuple of format (bucket_id, document_id)

In [23]:
def hash_bands(doc_band_set):
    bucket_docs = []
    for s in doc_band_set[1]:
        
        bucket_index = hash(str(sorted(s)))%n_lsh_buckets
        bucket_docs.append((bucket_index,doc_band_set[0]))
        
    return bucket_docs

Jaccard similarity implementation to compare sets of shingles

In [24]:
def jaccard_similarity(set_a,set_b):
    intersection = len(set_a.intersection(set_b))
    union = len(set_a.union(set_b))
    return intersection/union

In [25]:
hash_band_rdd = movie_signatures.mapValues(break_signatures_to_bands).flatMap(hash_bands)
hash_band_rdd.take(8)


[(4507, 23890098),
 (99, 23890098),
 (4828, 23890098),
 (836, 23890098),
 (271, 23890098),
 (143, 23890098),
 (1471, 23890098),
 (1625, 23890098)]

In [26]:
candidate_proximity_rdd = hash_band_rdd.groupByKey()
candidate_proximity_rdd.cache()

PythonRDD[19] at RDD at PythonRDD.scala:53

Function to generate candidate pairs from documents with a band hashed to a specific bucket

In [27]:
def compare_pairs(it):
    
    candidate_list = [x for x in iter(it)]
    pairs = []
    for i in range(len(candidate_list)):
        for j in range(i+1,len(candidate_list)):
            pairs.append((candidate_list[i],candidate_list[j]))
    
    return pairs

In [28]:
candidate_proximity_rdd = candidate_proximity_rdd.mapValues(compare_pairs).flatMap(lambda a: a[1]).distinct()

In [29]:
#example of candidate pairs
candidate_proximity_rdd.take(10)


[(23890098, 168554),
 (23890098, 30480836),
 (23890098, 28204476),
 (23890098, 15912738),
 (23890098, 15060240),
 (23890098, 1597802),
 (23890098, 8357964),
 (15912738, 15060240),
 (15912738, 1597802),
 (15912738, 8357964)]

Function to calculate jaccard similarity of 2 sets of signatures (2 movies)

In [30]:
def jaccard_signatures(sig1,sig2):
    a=0
    for i in range(k):
        if sig1[i]==sig2[i]:
            a+=1
    return a/k 
            

Function that compares 2 sets of signatures (2 movies)

In [31]:
def compare_movie_signatures(movie_1,movie_2):
    sig_1 = movie_signatures.lookup(movie_1)
    
    sig_2 = movie_signatures.lookup(movie_2)
    
    if len(sig_1)>0 and len(sig_2)> 0:
        return jaccard_signatures(list(sig_1[0].values()),list(sig_2[0].values()))
    else:
        return 0

Function that compares 2 sets of shingles (2 movies, exact comparison)

In [32]:
def compare_movie_shingles(movie_1,movie_2):
    sig_1 = movie_shingles.lookup(movie_1)
    sig_2 = movie_shingles.lookup(movie_2)
    if len(sig_1)>0 and len(sig_2)> 0:
        return jaccard_similarity(set(sig_1[0]),set(sig_2[0]))
    else:
        return 0

Function that given a movie and a similarity threshold (equal or higher to similarity_threshold achieved from number of bands and rows), returns from the candidate pairs, those whose signatures verify the required similarity
(exercise 2.2 when sim_percent=0.8)

In [33]:
def given_a_movie(movie_id,sim_percent):
    candidates = candidate_proximity_rdd.filter(lambda b: b[0]==movie_id or b[1]==movie_id).flatMap(lambda a: [x for x in list(a) if x!=movie_id]).collect()
    #print(candidates)
    similar = []
    for candidate in candidates:
        similarity_value = compare_movie_signatures(movie_id,candidate)
        if  (similarity_value >= sim_percent) and (similarity_value <=0.98):
        
            similar.append(candidate)
    return similar

Function that evaluates the results of function given_a_movie, by calculating the percentage of false positives, false negatives, true positives and true negatives (exercise 2.3)

A smaller subset of the dataset is recommended to run this function

In [34]:
def evaluate_result(movie_id, sim_percent):
    false_positives=0
    false_negatives=0
    true_positives=0
    true_negatives=0
    similar_movies = given_a_movie(movie_id,sim_percent)
    left_out_movies = movie_signatures.keys().filter(lambda a: (not a in similar_movies) and a!=movie_id).collect()
    for f_movie in left_out_movies:
        if compare_movie_signatures(movie_id, f_movie) >= sim_percent:
            false_negatives+=1
        else:
            true_negatives+=1
    for p_movie in similar_movies:
        if compare_movie_shingles(movie_id,p_movie)>=sim_percent:
            true_positives+=1
        else:
            false_positives+=1
    total = false_positives + false_negatives + true_positives + true_negatives
    print("false positives: ", false_positives, " ",100*false_positives/total,"%")
    print("false negatives: ", false_negatives, " ",100*false_negatives/total,"%")
    print("true positives: ", true_positives, " ",100*true_positives/total,"%")
    print("true negatives: ", true_negatives, " ",100*true_negatives/total,"%")

In [35]:
candidate_proximity_rdd.take(10)

[(23890098, 168554),
 (23890098, 30480836),
 (23890098, 28204476),
 (23890098, 15912738),
 (23890098, 15060240),
 (23890098, 1597802),
 (23890098, 8357964),
 (15912738, 15060240),
 (15912738, 1597802),
 (15912738, 8357964)]

Given that every run will have different movies (because of random sample), since it was taking hours to run the full dataset, we use some of the values above to check our functions

it is very unlikely that in such a small sample, many significantly similar movies occur, so instead, we hand picked a small dataset where we were sure significantly similar movies appeared, and even produced our own entries just for testing, and the algorithm performed well. This dataset is present in the file smaller_plots.txt

Testing with custom entries, copied from original plots, changed to various degrees and added at the end of the file

using smaller_plots.txt

In [36]:
compare_movie_signatures(14168925, 1416892533)

0.89

In [37]:
compare_movie_shingles(14168925, 1416892533)

0.8050314465408805

In [38]:
compare_movie_signatures(358886999, 35888695)

0.91

In [39]:
compare_movie_shingles(358886999, 35888695)

0.9371643394199786

In [40]:
similar_movies = given_a_movie(1416892533,0.8)
print("similar to: ",get_movie_title(1416892533))
for movie in similar_movies:
    print(movie,get_movie_title(movie))

similar to:  Custom movie or error
14168925 Total Corruption


In [41]:
similar_movies = given_a_movie(358886999,0.8)
print("similar to: ",get_movie_title(358886999))
for movie in similar_movies:
    print(movie,get_movie_title(movie))


similar to:  Custom movie or error
35888695 Yaaran Naal Baharan
35888695113 Custom movie or error
35888695112 Custom movie or error
35888695222 Custom movie or error


In [42]:
evaluate_result(35888695,0.8)

false positives:  0   0.0 %
false negatives:  0   0.0 %
true positives:  4   0.3853564547206166 %
true negatives:  1034   99.61464354527938 %


In [43]:
sc.stop()